In [1]:
import numpy as np

# Import PyTorch
import torch

# Import TensorLy
import tensorly as tl
from tensorly.tucker_tensor import tucker_to_tensor
from tensorly.random import check_random_state

Using numpy backend.


In [2]:
tl.set_backend('pytorch')

Using pytorch backend.


Make the results reproducible by fixing the random seed

In [3]:
random_state = 1234
rng = check_random_state(random_state)
#device = 'cuda:8'
device = 'cpu'

Define a random tensor which we will try to decompose. 

In [4]:
shape = [5, 5, 5]
tensor = tl.tensor(rng.random_sample(shape), device=device, requires_grad=True)

Initialise a random Tucker decomposition of that tensor



In [5]:
ranks = [5, 5, 5]
core = tl.tensor(rng.random_sample(ranks), device=device, requires_grad=True)
factors = [tl.tensor(rng.random_sample((tensor.shape[i], ranks[i])),
                 device=device, requires_grad=True) for i in range(tl.ndim(tensor))]


Now we just iterate through the training loop and backpropagate...



In [6]:
n_iter = 10000
lr = 0.00005
penalty = 0.1

optimizer = torch.optim.Adam([core]+factors, lr=lr)
# [core, factors[0], factors[1], ...]

for i in range(1, n_iter):
    # Important: do not forget to reset the gradients
    optimizer.zero_grad()

    # Reconstruct the tensor from the decomposed form
    rec = tucker_to_tensor(core, factors)

    # squared l2 loss
    loss = tl.norm(rec - tensor, 2)

    # squared l2 penalty on the factors of the decomposition
    for f in factors:
        loss = loss + penalty * f.pow(2).sum()

    loss.backward()
    optimizer.step()

    if i % 1000 == 0:
        rec_error = tl.norm(rec.data - tensor.data, 2)/tl.norm(tensor.data, 2)
        print("Epoch {},. Rec. error: {}".format(i, rec_error))


Epoch 1000,. Rec. error: 12.333837509155273
Epoch 2000,. Rec. error: 8.580610275268555
Epoch 3000,. Rec. error: 5.916234493255615
Epoch 4000,. Rec. error: 4.007179260253906
Epoch 5000,. Rec. error: 2.6491668224334717
Epoch 6000,. Rec. error: 1.7086536884307861
Epoch 7000,. Rec. error: 1.0960273742675781
Epoch 8000,. Rec. error: 0.7418539524078369
Epoch 9000,. Rec. error: 0.5647979378700256


# Now a CP decomposition :)

In [7]:
# Create random factors
factors = [tl.tensor(rng.random_sample((s, 5)),
                     device=device,
                     requires_grad=True)\
           for s in shape]

If you are not familiar with list comprehension, note that this is equivalent to writing:

In [8]:
factors = []
for s in shape:
    factors.append(tl.tensor(rng.random_sample((s, 5)),
                     device=device,
                     requires_grad=True))

In [9]:
[f.shape for f in factors]

[torch.Size([5, 5]), torch.Size([5, 5]), torch.Size([5, 5])]

In [10]:
# Optimise them...
n_iter = 10000
lr = 0.00005
penalty = 0.1

optimizer = torch.optim.Adam(factors, lr=lr)

for i in range(1, n_iter):
    optimizer.zero_grad()

    # Reconstruct the tensor from the decomposed form
    rec = tl.kruskal_to_tensor(factors)

    # squared l2 loss
    loss = tl.norm(tensor - rec, 2)

    # squared l2 penalty on the factors of the decomposition
    for f in factors:
        loss = loss + penalty * f.pow(2).sum()

    loss.backward()
    optimizer.step()

    if i % 1000 == 0:
        rec_error = tl.norm(rec.data - tensor.data, 2)/tl.norm(tensor.data, 2)
        print("Epoch {},. Rec. error: {}".format(i, rec_error))


Epoch 1000,. Rec. error: 0.637516438961029
Epoch 2000,. Rec. error: 0.5492339134216309
Epoch 3000,. Rec. error: 0.4935280978679657
Epoch 4000,. Rec. error: 0.45761317014694214
Epoch 5000,. Rec. error: 0.4335983991622925
Epoch 6000,. Rec. error: 0.41660022735595703
Epoch 7000,. Rec. error: 0.4029829502105713
Epoch 8000,. Rec. error: 0.39038383960723877
Epoch 9000,. Rec. error: 0.3780127465724945
